## Descripción del Agente de Planificación de Viajes

Este agente utiliza **LangChain** y el modelo **Google Gemini‑2.0‑flash** para orquestar varias subtareas de planificación de viaje. 

Cada subtarea (o “dimensión”) está implementada como una herramienta (BaseTool) independiente, lo que le da al agente gran **modularidad** y **flexibilidad**.

### Componentes principales

- **LLM**  
  ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)  
- **RateLimiter**  
  Controla hasta 15 llamadas/minuto para no exceder cuotas.  
- **PromptTemplates & LLMChains**  
  - chain_extraccion → Extrae en JSON los datos del formulario.  
  - chain_migratoria & chain_migratoria_final → Requisitos migratorios.  
  - chain_turismo & chain_turismo_final → Clima, temporada y eventos.  
  - chain_costos & chain_costos_final → Estimación de costos (incluye tiquetes).  
  - chain_presupuesto → Compara costos vs. presupuesto y sugiere recortes.  
  - chain_itinerario → Genera el itinerario final.  
- **Herramientas personalizadas**  
  1. ExtractionTool  
  2. MigratoriaTool  
  3. TurismoTool  
  4. CostosTool  
  5. PresupuestoTool  
  6. ItinerarioTool  
- **Agente LangChain**  
  Inicializado con AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True y return_intermediate_steps=True.

### Flujo de ejecución

1. **Ingreso de datos**  
   El usuario completa el formulario (widgets de IPython).  
2. **Construcción de la consulta**  
   Se genera un texto unificado (o “prompt”) con todos los campos.  
3. **Planificación por el agente**  
   El agente recibe el prompt y en un bucle **ReAct** decide qué herramienta invocar.  
4. **Ejecución de herramientas**  
   - ExtractionTool convierte el prompt en JSON estructurado.  
   - Cada dimensión usa **enriquecer_con_internet()** para:
     1. Crear contexto con LLM.  
     2. Generar consulta web (DDGS).  
     3. Recopilar resultados y devolver la respuesta final.  
5. **Itinerario final**  
   ItinerarioTool combina todas las salidas en un plan de viaje detallado.  
6. **Logging y exportación**  
   Cada paso se registra en llm_log y al final se exporta a Excel.

![Diagrama de flujo (Mermaid)](media\mermaid.png)

![Formulario de entrada](media\formulario_entrada.png)

# Construcción

### Configuración de entorno y logging

- **Carga de variables**: `load_dotenv()` importa las credenciales del archivo `.env`.  
- **Registro de interacciones**:  
  - `llm_log`: lista que almacena cada llamada al LLM.  
  - `log_llm(nombre_cadena, prompt, respuesta)`: añade timestamp, nombre de la cadena, prompt y respuesta.  
- **Exportación a Excel**:  
  `exportar_llm_log_a_excel()` convierte `llm_log` en un DataFrame ordenado por fecha y lo guarda en `llm_log.xlsx`.  

In [1]:
from dotenv import load_dotenv
import os
# Carga las variables del archivo .env
load_dotenv()

from datetime import datetime

llm_log = []

def log_llm(nombre_cadena: str, prompt_text: str, respuesta: str):
    llm_log.append({
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "nombre_cadena": nombre_cadena,
        "prompt": prompt_text,
        "respuesta": respuesta
    })
    
import pandas as pd

def exportar_llm_log_a_excel(nombre_archivo="llm_log.xlsx"):
    if not llm_log:
        print("No hay registros para exportar.")
        return
    
    df = pd.DataFrame(llm_log)
    df = df.sort_values("timestamp")  # Asegura orden cronológico
    df.to_excel(nombre_archivo, index=False)
    print(f"✅ Log exportado exitosamente a: {nombre_archivo}")

### Interfaz de usuario con IPyWidgets

- **Encabezado HTML**: bloque `HTML` personalizado con título, subtítulo y estilos de Davivienda.  
- **Función `valor_o_indiferente`**: retorna el valor ingresado o “Indiferente” si está vacío.  
- **Campos del formulario**: widgets de texto y textarea para origen, destino, fecha, duración, tipo de viaje, acompañantes, presupuesto y expectativas.  
- **Botón de envío**: `submit_button` ejecuta `on_submit_clicked` e imprime los valores procesados en `output`.  
- **Layouts**: uso de `VBox` y `Layout` para estructurar el formulario con contenedores internos (blanco) y externos (fondo rojo).  
- **Función `construir_consulta()`**: genera un string concatenado con todos los campos del formulario, listo para el agente.  

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

# Bloque HTML para el título con estilos personalizados
header_html = """
<div style="background-color:#EB001F; padding:20px; text-align:center; margin-bottom:20px;">
  <h1 style="color:white; margin:0;">DaviTravel - Davivienda</h1>
  <h3 style="color:white; margin-top:5px;">Su viaje puede estar en el lugar equivocado, planéelo con DaviTravel</h3>
  <p style="font-size:14px; color:white; margin-top:10px;">
    Complete la siguiente información para diseñar el plan de viaje perfecto.
  </p>
</div>
"""
display(HTML(header_html))

# Función auxiliar para asignar "Indiferente" si el campo está vacío
def valor_o_indiferente(valor):
    return valor if valor.strip() != "" else "Indiferente"

# Layout base para los widgets
layout_text = widgets.Layout(width='600px')
layout_area = widgets.Layout(width='600px', height='100px')

# Campos del formulario
origen = widgets.Text(
    value='',
    placeholder='Ej: "Bogotá" o "Bogotá, Colombia"',
    description='¿Desde dónde viaja?',
    style={'description_width': '200px'},
    layout=layout_text
)
origen_explicacion = widgets.HTML(
    value='<i>Describa el punto de partida del viaje. Ej: "Bogotá, Colombia".</i>'
)

destino = widgets.Text(
    value='',
    placeholder='Ej: "Cartagena" o "Torre Eiffel, París, Francia"',
    description='¿Hacia dónde se dirige?',
    style={'description_width': '200px'},
    layout=layout_text
)
destino_explicacion = widgets.HTML(
    value='<i>Mencione el destino o destinos deseados. Ej: "Cartagena" o "Torre Eiffel, París".</i>'
)

fecha = widgets.Text(
    value='',
    placeholder='Ej: "a mediados de julio", "en agosto" o "el próximo año"',
    description='¿Cuándo viajar?',
    style={'description_width': '200px'},
    layout=layout_text
)
fecha_explicacion = widgets.HTML(
    value='<i>Indique el periodo en el que se planea el viaje, ya sea con fechas específicas o términos generales.</i>'
)

duracion = widgets.Text(
    value='',
    placeholder='Ej: "entre 2 semanas y un mes" o "10 días"',
    description='Duración del viaje:',
    style={'description_width': '200px'},
    layout=layout_text
)
duracion_explicacion = widgets.HTML(
    value='<i>Especifique la duración del viaje, por ejemplo, "entre 2 semanas y un mes".</i>'
)

tipo_viaje = widgets.Text(
    value='',
    placeholder='Ej: "un viaje cultural con descanso", "una aventura urbana", "un recorrido de negocios innovador" o "mixto"',
    description='Tipo de viaje:',
    style={'description_width': '200px'},
    layout=layout_text
)
tipo_viaje_explicacion = widgets.HTML(
    value='<i>Describa el enfoque del viaje. Por ejemplo, "un viaje cultural con descanso" o "una aventura urbana".</i>'
)

perfil_viajero = widgets.Text(
    value='',
    placeholder='Ej: "viaja solo", "con familia", "con amigos", "con pareja y compañeros de trabajo", etc.',
    description='¿Con quién viaja?',
    style={'description_width': '200px'},
    layout=layout_text
)
perfil_viajero_explicacion = widgets.HTML(
    value='<i>Indique quiénes acompañan al viajero. Por ejemplo, "viaja solo" o "con familia y amigos".</i>'
)

presupuesto = widgets.Text(
    value='',
    placeholder='Ej: "alrededor de 1500000 COP"',
    description='Presupuesto (COP):',
    style={'description_width': '200px'},
    layout=layout_text
)
presupuesto_explicacion = widgets.HTML(
    value='<i>Escriba el presupuesto en pesos colombianos. Si se deja vacío, se interpretará como "Indiferente".</i>'
)

expectativas = widgets.Textarea(
    value='',
    placeholder='Ej: "Desea combinar visitas a lugares emblemáticos con experiencias auténticas y gastronomía local"',
    description='Expectativas adicionales:',
    style={'description_width': '200px'},
    layout=layout_area
)
expectativas_explicacion = widgets.HTML(
    value='<i>Comparta cualquier detalle o preferencia adicional respecto al viaje.</i>'
)

# Botón para enviar la información
submit_button = widgets.Button(
    description='Enviar Información',
    button_style='success'
)

output = widgets.Output()

def on_submit_clicked(b):
    with output:
        output.clear_output()
        print("Información del Viaje:")
        print("Desde dónde viaja:", valor_o_indiferente(origen.value))
        print("Hacia dónde se dirige:", valor_o_indiferente(destino.value))
        print("¿Cuándo viajar:", valor_o_indiferente(fecha.value))
        print("Duración del viaje:", valor_o_indiferente(duracion.value))
        print("Tipo de viaje:", valor_o_indiferente(tipo_viaje.value))
        print("Con quién viaja:", valor_o_indiferente(perfil_viajero.value))
        print("Presupuesto (COP):", valor_o_indiferente(presupuesto.value))
        print("Expectativas adicionales:", valor_o_indiferente(expectativas.value))

submit_button.on_click(on_submit_clicked)

# Organiza el formulario en un contenedor blanco interno
form_inner = widgets.VBox([
    widgets.VBox([origen, origen_explicacion]),
    widgets.VBox([destino, destino_explicacion]),
    widgets.VBox([fecha, fecha_explicacion]),
    widgets.VBox([duracion, duracion_explicacion]),
    widgets.VBox([tipo_viaje, tipo_viaje_explicacion]),
    widgets.VBox([perfil_viajero, perfil_viajero_explicacion]),
    widgets.VBox([presupuesto, presupuesto_explicacion]),
    widgets.VBox([expectativas, expectativas_explicacion]),
    submit_button,
    output
], layout=widgets.Layout(
    border='2px solid #fff',
    background_color='#ffffff',
    padding='20px'
))

# Organiza el formulario completo en un contenedor externo (fondo rojo)
form_outer = widgets.VBox([form_inner], layout=widgets.Layout(
    background_color='#ed1c27',
    padding='20px',
    width='100%',
    align_items='center'
))

display(form_outer)

def construir_consulta() -> str:
    """
    Construye la cadena de consulta a partir de los valores ingresados en el formulario.
    Si algún campo está vacío, se considera "Indiferente".
    """
    consulta = (
        "Desde dónde viaja: " + valor_o_indiferente(origen.value) + "; " +
        "Hacia dónde se dirige: " + valor_o_indiferente(destino.value) + "; " +
        "Cuándo viajar: " + valor_o_indiferente(fecha.value) + "; " +
        "Duración del viaje: " + valor_o_indiferente(duracion.value) + "; " +
        "Tipo de viaje: " + valor_o_indiferente(tipo_viaje.value) + "; " +
        "Con quién viaja: " + valor_o_indiferente(perfil_viajero.value) + "; " +
        "Presupuesto: " + valor_o_indiferente(presupuesto.value) + "; " +
        "Expectativas adicionales: " + valor_o_indiferente(expectativas.value)
    )
    return consulta

In [4]:
import time
from collections import deque
import os
import requests

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import GoogleSearchAPIWrapper

### Control de tasa de llamadas

- **Clase `RateLimiter`**: limita a `max_calls` ejecuciones en un periodo de `period` segundos.  
- **Método `wait()`**:  
  1. Elimina registros de llamadas fuera del periodo.  
  2. Si se alcanzó el límite, espera el tiempo restante.  
  3. Registra el timestamp de la llamada actual.  
- **Instancia `limiter`**: configurada para 15 llamadas cada 60 segundos.  

In [ ]:
class RateLimiter:
    def __init__(self, max_calls: int, period: float):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
    def wait(self):
        now = time.monotonic()
        while self.calls and now - self.calls[0] > self.period:
            self.calls.popleft()
        if len(self.calls) >= self.max_calls:
            to_wait = self.period - (now - self.calls[0])
            time.sleep(to_wait)
            now = time.monotonic()
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()
        self.calls.append(now)

limiter = RateLimiter(max_calls=15, period=60.0)

### Inicialización del LLM

- **gemini_llm**: instancia de Google Gemini 2.0‑flash con temperatura 0.3.  
- La clave `GOOGLE_API_KEY` se carga desde las variables de entorno.  


In [ ]:
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0.3
)

### 🔍 Extracción de datos del formulario

- **PromptTemplate** `prompt_extraccion`: instrucción para extraer origen, destino, fecha, duración, tipo de viaje, acompañantes, presupuesto y expectativas en JSON corto.  
- **LLMChain** `chain_extraccion`: une `gemini_llm` con `prompt_extraccion` para procesar la extracción.

In [7]:
# ========================================
# 🔍 EXTRACCIÓN DE DATOS DEL FORMULARIO
# ========================================

prompt_extraccion = PromptTemplate(
    input_variables=["datos_formulario"],
    template=(
        "Extraiga los siguientes datos del texto: origen, destino, fecha, duración del viaje, tipo de viaje, acompañantes, presupuesto y expectativas.\n\n"
        "Texto:\n{datos_formulario}\n\nResponda en formato JSON corto."
    )
)
chain_extraccion = LLMChain(llm=gemini_llm, prompt=prompt_extraccion)

C:\Users\fcalv\AppData\Local\Temp\ipykernel_29112\2322308999.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_extraccion = LLMChain(llm=gemini_llm, prompt=prompt_extraccion)


### 🌍 Dimensión Migratoria

- **`prompt_migratoria`**: genera una consulta web en español (una línea) para buscar requisitos migratorios.  
- **`chain_migratoria`**: LLMChain que usa `prompt_migratoria`.  
- **`prompt_consulta_migratoria`**: a partir del contexto, crea una consulta breve y específica para buscar requisitos oficiales.  
- **`prompt_migratoria_final`**: procesa los resultados web y lista visa, vacunas u otros documentos necesarios.  
- **`chain_migratoria_final`**: LLMChain que usa `prompt_migratoria_final`.  

In [8]:
# ========================================
# 🌍 DIMENSIÓN MIGRATORIA
# ========================================

prompt_migratoria = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Dado este requerimiento de viaje: {info_requerimientos}, "
        "genere una sola consulta de búsqueda en español (máximo una línea) para buscar los requisitos migratorios en internet."
    )
)
chain_migratoria = LLMChain(llm=gemini_llm, prompt=prompt_migratoria)

prompt_consulta_migratoria = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Con base en este contexto migratorio generado por el sistema:\n\n{contexto}\n\n"
        "Escriba una consulta de búsqueda breve y específica en español para internet. "
        "La consulta debe buscar requisitos migratorios oficiales para ciudadanos colombianos hacia el país destino. "
        "No escriba explicaciones, solo la consulta en una línea."
    )
)

prompt_migratoria_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en esta información y los resultados web, indique los requisitos migratorios para el viaje:\n\n"
        "{info_requerimientos}\n\n"
        "Sea claro y conciso. Liste si se requiere visa, vacunas u otros documentos."
    )
)
chain_migratoria_final = LLMChain(llm=gemini_llm, prompt=prompt_migratoria_final)


### 🏖️ Dimensión Turismo

- **`prompt_turismo`**: PromptTemplate que describe clima, temporadas y eventos relevantes en el destino usando viñetas.  
- **`chain_turismo`**: LLMChain que utiliza `prompt_turismo`.  
- **`prompt_consulta_turismo`**: Genera una consulta corta y concreta para buscar clima, eventos o temporadas en internet.  
- **`prompt_turismo_final`**: Resume clima, eventos y temporada a partir de los resultados web, usando viñetas.  
- **`chain_turismo_final`**: LLMChain que utiliza `prompt_turismo_final`.  

In [9]:
# ========================================
# 🏖️ DIMENSIÓN TURISMO
# ========================================

prompt_turismo = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Describa brevemente el clima, las temporadas turísticas (alta o baja) y eventos relevantes en el destino según esta información:\n\n"
        "{info_requerimientos}\n\nUse viñetas si es posible. Sea claro y evite detalles innecesarios."
    )
)
chain_turismo = LLMChain(llm=gemini_llm, prompt=prompt_turismo)

prompt_consulta_turismo = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Según este contexto turístico:\n\n{contexto}\n\n"
        "Genere una consulta de búsqueda concreta y corta en español para internet. "
        "Debe enfocarse en eventos turísticos, clima o temporadas en el destino en las fechas del viaje. "
        "No incluya explicaciones, solo la consulta."
    )
)

prompt_turismo_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Utilice la siguiente información y resultados de búsqueda para resumir el clima, eventos y temporada turística del destino:\n\n"
        "{info_requerimientos}\n\nUse viñetas si es posible. Sea concreto."
    )
)
chain_turismo_final = LLMChain(llm=gemini_llm, prompt=prompt_turismo_final)

### 💸 Dimensión Costos

- **`prompt_costos`**: define la estimación diaria en COP de transporte (incluye tiquetes), alojamiento y comida.  
- **`prompt_consulta_costos`**: genera una consulta breve para buscar precios promedio de tiquetes (aéreos o terrestres), transporte local, alojamiento y comida.  
- **`prompt_costos_final`**: formatea la respuesta final con el desglose de costos diarios por concepto.  
- **`chain_costos`**: LLMChain que ejecuta `prompt_costos`.  
- **`chain_costos_final`**: LLMChain que ejecuta `prompt_costos_final`.  

In [31]:
# ========================================
# 💸 DIMENSIÓN COSTOS
# ========================================

# 1. Prompt para estimación total y por persona
prompt_costos = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en la información del viaje y el número de viajeros, estime:\n"
        "- Costo total diario en COP de transporte (tiquetes incluidos), alojamiento y comida.\n"
        "- Costo diario por persona en COP para cada concepto.\n\n"
        "Datos:\n{info_requerimientos}\n\n"
        "Presente valores totales y por persona."
    )
)

# 2. Prompt para consulta web de costos generales
prompt_consulta_costos = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Resumen de contexto sobre costos estimados:\n\n{contexto}\n\n"
        "Genere una consulta en español (una línea) para buscar en internet:\n"
        "- Precio promedio de transporte local, alojamiento y comida en el destino.\n"
        "No incluya explicaciones, solo la consulta."
    )
)

# 3. Prompt para consulta web específica de tiquetes
prompt_consulta_tiquetes = PromptTemplate(
    input_variables=["contexto"],
    template=(
        "Según este contexto de viaje:\n\n{contexto}\n\n"
        "Genere una consulta breve en español para buscar en internet el precio promedio "
        "de tiquetes (aéreos o terrestres) desde el origen al destino indicado. Solo la consulta."
    )
)

# 4. Prompt final que integra todo
prompt_costos_final = PromptTemplate(
    input_variables=["info_requerimientos"],
    template=(
        "Con base en la información y resultados web, calcule el **costo total del viaje** en COP, desglosado por:\n"
        "- Transporte (tiquetes incluidos): total y por persona.\n"
        "- Alojamiento: total y por persona.\n"
        "- Alimentación: total y por persona.\n\n"
        "{info_requerimientos}\n\n"
        "Solo muestre valores numéricos con etiquetas claras."
    )
)

chain_costos = LLMChain(llm=gemini_llm, prompt=prompt_costos)
chain_costos_final = LLMChain(llm=gemini_llm, prompt=prompt_costos_final)

### 📊 Dimensión Presupuesto

- **`prompt_presupuesto`**: calcula el total de los costos estimados, lo compara con el presupuesto del usuario y, si excede, sugiere 1 o 2 formas de reducir gastos.  
- **`chain_presupuesto`**: LLMChain que ejecuta `prompt_presupuesto` usando `gemini_llm`.  

In [32]:
# ========================================
# 📊 DIMENSIÓN PRESUPUESTO
# ========================================

prompt_presupuesto = PromptTemplate(
    input_variables=["costos_totales", "info_usuario"],
    template=(
        "Tienes estos costos totales del viaje en COP:\n{costos_totales}\n\n"
        "Y esta información del usuario (incluye presupuesto):\n{info_usuario}\n\n"
        "1. Extrae el presupuesto del usuario.\n"
        "2. Compara el presupuesto con el **costo total**:\n"
        "   - Si el presupuesto ≥ costo total: confirma que cubre los gastos y muestra qué % representa cada concepto.\n"
        "   - Si el presupuesto < costo total: indica que NO es suficiente y sugiere 2 ajustes al plan.\n"
        "No hagas desglose del presupuesto si no alcanza."
    )
)
chain_presupuesto = LLMChain(llm=gemini_llm, prompt=prompt_presupuesto)

### 🧳 Dimensión Itinerario Final

- **`prompt_itinerario`**: PromptTemplate que recibe `info_migratoria`, `info_turismo`, `presupuesto_final` e `info_usuario` para generar un itinerario en cinco secciones:  
  1. Documentos y requisitos  
  2. Información general del destino  
  3. Presupuesto estimado  
  4. Itinerario diario sugerido  
  5. Recomendaciones finales  
- **`chain_itinerario`**: LLMChain que une `gemini_llm` con `prompt_itinerario` para producir el plan de viaje completo.  

In [38]:
# ========================================
# 🧳 ITINERARIO FINAL
# ========================================

prompt_itinerario = PromptTemplate(
    input_variables=["info_migratoria", "info_turismo", "presupuesto_final", "info_usuario"],
    template=(
        "¡Prepárese para vivir la experiencia de su vida con DaviTravel! "
        "A continuación encontrará un itinerario ideal para el viaje de sus sueños:\n\n"
        "0. **Preferencias de viaje:**\n"
        "{info_usuario}\n\n"
        "1. **Requisitos migratorios:**\n"
        "{info_migratoria}\n\n"
        "2. **Información turística:**\n"
        "{info_turismo}\n\n"
        "3. **Presupuesto y costos estimados:**\n"
        "{presupuesto_final}\n\n"
        "Organice el plan en estas secciones:\n\n"
        "I. **Documentos y requisitos**\n"
        "- Documentos imprescindibles para ingreso y salida.\n"
        "- Visa, vacunas u otros permisos necesarios.\n\n"
        "II. **Resumen del destino**\n"
        "- Clima en las fechas seleccionadas.\n"
        "- Temporada (alta o baja) y eventos especiales.\n\n"
        "III. **Desglose de presupuesto**\n"
        "- Transporte, alojamiento y alimentación en COP.\n"
        "- Indique si el plan se ajusta al presupuesto o sugiera ajustes.\n\n"
        "IV. **Itinerario diario**\n"
        "Para cada día, proponga:\n"
        "- Hasta 3 actividades destacadas.\n"
        "- Tipo de experiencia (cultural, gastronómica, aventura, descanso).\n"
        "- Horarios recomendados y ubicaciones clave.\n\n"
        "V. **Toques finales**\n"
        "- Consejos exclusivos según su perfil de viaje.\n"
        "- Recomendaciones de reservas anticipadas y transporte local.\n\n"
        "Haga que cada momento sea inolvidable y despierte la emoción de su próxima aventura."
    )
)
chain_itinerario = LLMChain(llm=gemini_llm, prompt=prompt_itinerario)

### Función de enriquecimiento con internet

- Importa `DDGS` de `duckduckgo_search` para realizar búsquedas web.  
- **`enriquecer_con_internet(...)`**:
  1. Genera contexto base con el LLM y lo registra.  
  2. Crea una consulta web a partir del contexto y la registra.  
  3. Realiza búsqueda en DuckDuckGo (hasta 5 resultados) y registra los enlaces.  
  4. Combina contexto base y resultados de internet.  
  5. Ejecuta la cadena final con el contexto enriquecido y registra la respuesta.  
- Devuelve el resultado final enriquecido con información de la web.  

In [34]:
from duckduckgo_search import DDGS

def enriquecer_con_internet(
    prompt_contextual: PromptTemplate, chain_contextual: LLMChain,
    prompt_consulta: PromptTemplate,
    prompt_final: PromptTemplate, chain_final: LLMChain,
    info_usuario: str, nombre_cadena: str
) -> str:
    """
    Genera contexto con el LLM, realiza búsqueda en internet y ejecuta una cadena final
    enriquecida, registrando logs detallados en cada paso.
    """
    # Paso 1: Generar contexto base con el modelo
    prompt_ctx = prompt_contextual.format(info_requerimientos=info_usuario)
    limiter.wait()
    contexto_base = chain_contextual.run({"info_requerimientos": info_usuario})
    log_llm(f"{nombre_cadena} - Contexto Base", prompt_ctx, contexto_base)

    # Paso 2: Generar la consulta de búsqueda usando el contexto generado
    consulta_prompt = prompt_consulta.format(contexto=contexto_base)
    limiter.wait()
    consulta_generada = gemini_llm.invoke(consulta_prompt).content.strip()
    consulta_simple = consulta_generada.replace("", "").split("\n")[0].strip()
    log_llm(f"{nombre_cadena} - Consulta Web", consulta_prompt, consulta_generada)

    # Paso 3: Ejecutar búsqueda en internet
    limiter.wait()
    resultados = []
    with DDGS() as ddgs:
        for r in ddgs.text(consulta_simple, max_results=5):
            resultados.append(f"{r['title']}: {r['href']}")
    contexto_internet = "\n".join(resultados)

    # Guardar los resultados de búsqueda como parte del contexto
    log_llm(f"{nombre_cadena} - Resultados Web", consulta_simple, contexto_internet)

    # Paso 4: Crear contexto combinado
    contexto_completo = f"{contexto_base}\n\nResultados de internet:\n{contexto_internet}"

    # Paso 5: Ejecutar la cadena final con el contexto enriquecido
    prompt_final_text = prompt_final.format(info_requerimientos=contexto_completo)
    limiter.wait()
    resultado = chain_final.run({"info_requerimientos": contexto_completo})
    log_llm(f"{nombre_cadena} - Resultado Final", prompt_final_text, resultado)

    return resultado

## Primera Versión

### Generación secuencial del plan de viaje

- **Función** `generar_plan_de_viaje(info_requerimientos: str) -> str`: en esta versión inicial el agente ejecuta todas las cadenas de forma secuencial dentro de una sola función.  
- **Rate limiting**: antes de cada llamada a LLM o búsqueda web se invoca `limiter.wait()` para respetar la cuota.  
- **Pasos**:
  1. **Migratoria**: `enriquecer_con_internet(...)` genera requisitos migratorios.  
  2. **Turismo**: idem para clima, temporadas y eventos.  
  3. **Costos**: idem para transporte (incluye tiquetes), alojamiento y comida.  
  4. **Presupuesto**: `chain_presupuesto.run(...)` compara costos vs. presupuesto y sugiere recortes.  
  5. **Itinerario**: `chain_itinerario.run(...)` combina todos los resultados en un itinerario detallado.  
- **Logging**: cada llamada guarda prompt y respuesta en `llm_log` mediante `log_llm`.  
- **Retorno**: devuelve el itinerario completo como string.  

In [ ]:
def generar_plan_de_viaje(info_requerimientos: str) -> str:
    limiter.wait()
    migratoria = enriquecer_con_internet(
        prompt_contextual=prompt_migratoria,
        chain_contextual=chain_migratoria,
        prompt_consulta=prompt_consulta_migratoria,
        prompt_final=prompt_migratoria_final,
        chain_final=chain_migratoria_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Información Migratoria"
    )

    limiter.wait()
    turismo = enriquecer_con_internet(
        prompt_contextual=prompt_turismo,
        chain_contextual=chain_turismo,
        prompt_consulta=prompt_consulta_turismo,
        prompt_final=prompt_turismo_final,
        chain_final=chain_turismo_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Información Turística"
    )

    limiter.wait()
    costos = enriquecer_con_internet(
        prompt_contextual=prompt_costos,
        chain_contextual=chain_costos,
        prompt_consulta=prompt_consulta_costos,
        prompt_final=prompt_costos_final,
        chain_final=chain_costos_final,
        info_usuario=info_requerimientos,
        nombre_cadena="Estimación de Costos"
    )

    limiter.wait()
    presupuesto_final = chain_presupuesto.run({"costos": costos})
    log_llm("Cálculo de Presupuesto", prompt_presupuesto.format(costos=costos), presupuesto_final)

    limiter.wait()
    itinerario = chain_itinerario.run({
    "info_migratoria": migratoria,
    "info_turismo": turismo,
    "presupuesto_final": presupuesto_final,
    "info_usuario": info_requerimientos
    })

    log_llm(
        "Itinerario Final",
        prompt_itinerario.format(
            info_migratoria=migratoria,
            info_turismo=turismo,
            presupuesto_final=presupuesto_final,
            info_usuario=info_requerimientos
        ),
        itinerario
    )
    
    return itinerario

### Herramienta personalizada `TravelPlannerTool`

- **Clase**: hereda de `BaseTool` para integrarse al agente LangChain.  
- **`name`**: `"TravelPlannerTool"`, identificador de la herramienta.  
- **`description`**: resumen de su función (“recibe una consulta y retorna un plan de viaje completo”).  
- **`_run(query: str) -> str`**:  
  1. `limiter.wait()` para respetar la cuota de llamadas.  
  2. Extrae datos del formulario con `chain_extraccion.run(...)`.  
  3. Registra la extracción en `llm_log` vía `log_llm`.  
  4. Llama a `generar_plan_de_viaje(info)` que ejecuta secuencialmente todas las cadenas (migratoria, turismo, costos, presupuesto e itinerario) y devuelve el itinerario final.  
- **`_arun`**: no implementado (solo síncrono).  
- **Contexto**: primera iteración del agente, encapsula todo el flujo secuencial en una única herramienta.  

In [ ]:
class TravelPlannerTool(BaseTool):
    name: str = "TravelPlannerTool"
    description: str = "Planificador de viajes: recibe una consulta y retorna un resumen completo del plan de viaje."
    def _run(self, query: str) -> str:
        limiter.wait()
        info = chain_extraccion.run({"datos_formulario": query})
        log_llm("Extracción de Requerimientos", prompt_extraccion.format(datos_formulario=query), info)
        return generar_plan_de_viaje(info)
    async def _arun(self, query: str) -> str:
        raise NotImplementedError

### Inicialización del Agente

- **`travel_planner_tool`**: instancia de la herramienta personalizada `TravelPlannerTool`.  
- **`agent`**: creado con `initialize_agent()` que recibe:
  - `tools=[travel_planner_tool]`  
  - `llm=gemini_llm`  
  - `agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION`  
  - `verbose=False`  
  - `return_intermediate_steps=True`  
- **Descripción**: el agente emplea el patrón ReAct para procesar la entrada del usuario y llamar a la herramienta única, devolviendo el itinerario generado.  

In [ ]:
from langchain.agents import initialize_agent

travel_planner_tool = TravelPlannerTool()
agent = initialize_agent(
    tools=[travel_planner_tool],
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    return_intermediate_steps=True
)

C:\Users\fcalv\AppData\Local\Temp\ipykernel_29112\2472800825.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### Integración con el formulario y ejecución final

- Genera la consulta unificada con `consulta_agente = construir_consulta()`.  
- Llama a `limiter.wait()` para respetar el rate limit.  
- Ejecuta el agente con `agent.invoke({"input": consulta_agente}, return_intermediate_steps=True)` y guarda el resultado en `result`.  
- Exporta los logs de LLM a Excel usando `exportar_llm_log_a_excel()`.  
- Extrae la última observación (`result["intermediate_steps"][-1][1]`), que contiene el itinerario final.  
- Imprime el itinerario en consola con formato y encabezado.  

In [ ]:
consulta_agente = construir_consulta()

limiter.wait()
result = agent.invoke({"input": consulta_agente}, return_intermediate_steps=True)
exportar_llm_log_a_excel()

# Extraer la última Observation (que contiene el resultado del itinerario final)
ultimo_observation = result["intermediate_steps"][-1][1]  # [1] es el observation

# Mostrar solo el itinerario generado
print("\n" + "=" * 60)
print("📦 ITINERARIO DETALLADO DEL VIAJE")
print("=" * 60)
print(ultimo_observation.strip())

C:\Users\fcalv\AppData\Local\Temp\ipykernel_29112\545450216.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  info = chain_extraccion.run({"datos_formulario": query})


✅ Log exportado exitosamente a: llm_log.xlsx

📦 ITINERARIO DETALLADO DEL VIAJE
De acuerdo, aquí tienes el itinerario detallado y conciso basado en la información proporcionada:

**El presupuesto del usuario es de 5,000,000 COP para un viaje de 4-5 días.**

**I. Documentos y requisitos:**

*   **Pasaporte:** Validez de al menos 3 meses superior a la fecha prevista de salida del espacio Schengen.
*   **Tiquete de regreso:** Demostración de salida de Italia (o espacio Schengen) dentro de 90 días.
*   **Justificación del viaje:** Reservas de hotel, itinerario.
*   **Solvencia económica:** Efectivo, tarjetas de crédito, extractos bancarios.
*   **Seguro de viaje:** Cobertura médica en Europa (altamente recomendable).
*   **ETIAS:** (A partir de 2025, fecha por confirmar).
*   **Vacunas:** No obligatorias, pero esquema de vacunación general al día.
*   **Visa:** No requerida para estancias de turismo de hasta 90 días.
*   **Formularios especiales:** No se mencionan formularios especiales, pe

## Segunda Versión

### Herramientas personalizadas para el agente modular

- **ExtractionTool**  
  Extrae en JSON los campos del formulario de viaje y registra la operación (usa `chain_extraccion`).

- **MigratoriaTool**  
  Genera requisitos migratorios enriquecidos con búsqueda web (usa `enriquecer_con_internet` con `chain_migratoria`).

- **TurismoTool**  
  Describe clima, temporadas y eventos turísticos con datos de internet (usa `enriquecer_con_internet` con `chain_turismo`).

- **CostosTool**  
  Estima costos diarios de transporte (incluye tiquetes), alojamiento y comida mediante búsqueda web (usa `enriquecer_con_internet` con `chain_costos`).

- **PresupuestoTool**  
  Calcula el total de costos, lo compara con el presupuesto del usuario y sugiere recortes (usa `chain_presupuesto`).

- **ItinerarioTool**  
  Recibe un JSON con `info_migratoria`, `info_turismo`, `presupuesto_final` e `info_usuario`, lo parsea y genera el itinerario final (usa `chain_itinerario`).

In [39]:
from langchain.tools import BaseTool

# 1. Extracción de requerimientos
class ExtractionTool(BaseTool):
    name: str = "extraer_requerimientos"
    description: str = "Extrae en JSON los campos del formulario de viaje."
    def _run(self, query: str) -> str:
        limiter.wait() 
        info = chain_extraccion.run({"datos_formulario": query})
        log_llm("Extracción de Requerimientos",
                prompt_extraccion.format(datos_formulario=query),
                info)
        return info
    async def _arun(self, query: str) -> str:
        raise NotImplementedError

# 2. Migratoria
class MigratoriaTool(BaseTool):
    name: str = "migratoria"
    description: str = "Devuelve los requisitos migratorios."
    def _run(self, info_requerimientos: str) -> str:
        limiter.wait() 
        return enriquecer_con_internet(
            prompt_contextual=prompt_migratoria,
            chain_contextual=chain_migratoria,
            prompt_consulta=prompt_consulta_migratoria,
            prompt_final=prompt_migratoria_final,
            chain_final=chain_migratoria_final,
            info_usuario=info_requerimientos,
            nombre_cadena="Información Migratoria"
        )
    async def _arun(self, info_requerimientos: str) -> str:
        raise NotImplementedError

# 3. Turismo
class TurismoTool(BaseTool):
    name: str = "turismo"
    description: str = "Devuelve clima, temporada y eventos turísticos."
    def _run(self, info_requerimientos: str) -> str:
        limiter.wait()
        return enriquecer_con_internet(
            prompt_contextual=prompt_turismo,
            chain_contextual=chain_turismo,
            prompt_consulta=prompt_consulta_turismo,
            prompt_final=prompt_turismo_final,
            chain_final=chain_turismo_final,
            info_usuario=info_requerimientos,
            nombre_cadena="Información Turística"
        )
    async def _arun(self, info_requerimientos: str) -> str:
        raise NotImplementedError

# 4. Costos
class CostosTool(BaseTool):
    name: str = "costos"
    description: str = "Estima costos totales y por persona de transporte (tiquetes), alojamiento y comida, con consulta adicional de tiquetes."
    def _run(self, info_requerimientos: str) -> str:
        # 6.1 Estimación general de costos
        limiter.wait()
        contexto_general = enriquecer_con_internet(
            prompt_contextual=prompt_costos,
            chain_contextual=chain_costos,
            prompt_consulta=prompt_consulta_costos,
            prompt_final=prompt_costos_final,
            chain_final=chain_costos_final,
            info_usuario=info_requerimientos,
            nombre_cadena="Estimación de Costos"
        )

        # 6.2 Consulta adicional de tiquetes
        limiter.wait()
        consulta_tiq = prompt_consulta_tiquetes.format(contexto=info_requerimientos)
        consulta_tiq = gemini_llm.invoke(consulta_tiq).content.strip()
        log_llm("Consulta Tiquetes", consulta_tiq, "")

        limiter.wait()
        resultados_tiq = []
        with DDGS() as ddgs:
            for r in ddgs.text(consulta_tiq, max_results=5):
                resultados_tiq.append(f"{r['title']}: {r['href']}")
        contexto_tiquetes = "\n".join(resultados_tiq)
        log_llm("Resultados Tiquetes", consulta_tiq, contexto_tiquetes)

        # 6.3 Combinar y devolver
        return (
            contexto_general
            + "\n\nCosto real de tiquetes (enlaces):\n"
            + contexto_tiquetes
        )

    async def _arun(self, info_requerimientos: str) -> str:
        return self._run(info_requerimientos)

# 5. Presupuesto
class PresupuestoTool(BaseTool):
    name: str = "presupuesto"
    description: str = (
        "Recibe un JSON con 'costos_totales' e 'info_usuario', compara presupuesto vs costo total y sugiere ajustes."
    )
    def _run(self, tool_input: str) -> str:
        try:
            datos = json.loads(tool_input)
        except json.JSONDecodeError:
            raise ValueError("PresupuestoTool: la entrada debe ser un JSON con 'costos_totales' e 'info_usuario'")
        costos_totales = datos.get("costos_totales")
        info_usuario = datos.get("info_usuario")
        if costos_totales is None or info_usuario is None:
            raise ValueError("PresupuestoTool: faltan 'costos_totales' o 'info_usuario'")
        resultado = chain_presupuesto.run({
            "costos_totales": costos_totales,
            "info_usuario": info_usuario
        })
        log_llm(
            "Cálculo de Presupuesto",
            prompt_presupuesto.format(costos_totales=costos_totales, info_usuario=info_usuario),
            resultado
        )
        return resultado

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

# 6. Itinerario
import json
from langchain.tools import BaseTool

class ItinerarioTool(BaseTool):
    name: str = "itinerario"
    description: str = (
        "Recibe un string que contiene un JSON con las claves "
        "info_migratoria, info_turismo, presupuesto_final e info_usuario "
        "y devuelve el itinerario detallado."
    )

    def _run(self, tool_input: str) -> str:
        # 1) Extraer el JSON del input (busca el primer { ... } balanceado)
        start = tool_input.find("{")
        if start == -1:
            raise ValueError("ItinerarioTool: no se encontró ningún JSON en la entrada")
        depth = 0
        end = None
        for i, ch in enumerate(tool_input[start:], start):
            if ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    end = i + 1
                    break
        if end is None:
            raise ValueError("ItinerarioTool: JSON mal balanceado en la entrada")

        json_str = tool_input[start:end]
        try:
            datos = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"ItinerarioTool: error al parsear JSON extraído: {e}")

        # 2) Validar claves requeridas
        for key in ("info_migratoria", "info_turismo", "presupuesto_final", "info_usuario"):
            if key not in datos:
                raise ValueError(f"ItinerarioTool: falta la clave '{key}' en el JSON de entrada")

        # 3) Ejecutar el chain con los valores extraídos
        itinerario = chain_itinerario.run({
            "info_migratoria": datos["info_migratoria"],
            "info_turismo": datos["info_turismo"],
            "presupuesto_final": datos["presupuesto_final"],
            "info_usuario": datos["info_usuario"]
        })

        # 4) Registrar y devolver
        log_llm(
            "Itinerario Final",
            prompt_itinerario.format(
                info_migratoria=datos["info_migratoria"],
                info_turismo=datos["info_turismo"],
                presupuesto_final=datos["presupuesto_final"],
                info_usuario=datos["info_usuario"]
            ),
            itinerario
        )
        return itinerario

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)


### Inicialización del agente modular

- **Herramientas**: `ExtractionTool`, `MigratoriaTool`, `TurismoTool`, `CostosTool`, `PresupuestoTool`, `ItinerarioTool`.  
- **Agente**:  
  - `initialize_agent(...)` recibe la lista de herramientas y el LLM `gemini_llm`.  
  - Usa `AgentType.ZERO_SHOT_REACT_DESCRIPTION` para el patrón ReAct.  
  - `verbose=True` muestra en consola qué herramienta selecciona en cada paso.  

In [40]:
from langchain.agents import initialize_agent, AgentType

tools = [
    ExtractionTool(),
    MigratoriaTool(),
    TurismoTool(),
    CostosTool(),
    PresupuestoTool(),
    ItinerarioTool(),
]

agent = initialize_agent(
    tools=tools,
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True#,                   # para ver en consola qué herramienta escoge
    #return_intermediate_steps=True  # si quieres los pasos intermedios
)

### Ejemplo de invocación

- Reinicia `llm_log` para registrar desde cero.  
- Genera la consulta con `construir_consulta()`.  
- Llama a `limiter.wait()` para respetar el rate limit.  
- Ejecuta `agent.invoke({"input": consulta})` y guarda la respuesta en `result`.  
- Muestra el itinerario con `print(result["output"])`.  
- Exporta todos los logs a Excel con `exportar_llm_log_a_excel()`.  

In [41]:
llm_log = []
consulta = construir_consulta()
limiter.wait()
result = agent.invoke({"input": consulta})
print(result["output"])
exportar_llm_log_a_excel()



> Entering new AgentExecutor chain...
Okay, I need to create a travel itinerary based on the user's input. First, I need to extract the travel requirements from the user's input.
Action: extraer_requerimientos
Action Input: Desde dónde viaja: pereira; Hacia dónde se dirige: roma, italia; Cuándo viajar: verano europa; Duración del viaje: 4 o 5 días; Tipo de viaje: cultural y gastronomico; Con quién viaja: solo; Presupuesto: 5 millones; Expectativas adicionales: quiero comer la mejor pasta
Observation: ```json
{
  "origen": "pereira",
  "destino": "roma, italia",
  "fecha": "verano europa",
  "duracion": "4 o 5 días",
  "tipo_viaje": "cultural y gastronomico",
  "acompanantes": "solo",
  "presupuesto": "5 millones",
  "expectativas": "quiero comer la mejor pasta"
}
```
Thought:Now that I have the travel requirements, I need to check the migratory requirements for traveling from Pereira to Rome, Italy.
Action: migratoria
Action Input: {"origen": "pereira", "destino": "roma, italia"}
Obs

# Tercera Versión

### Integración de alianzas Davivienda

- **ChatPromptTemplate**  
  - Mensaje **system**: “Usted es DaviTravel…” fija tono en español, tercera persona, entusiasta y cercano.  
  - Mensaje **human**: toma el itinerario y la lista de alianzas, e inserta recomendaciones de productos Davivienda en cada sección.

- **LLMChain** `chain_alianzas`  
  Une `gemini_llm` con el `prompt_alianzas` para generar el itinerario enriquecido.

- **Clase `AlianzasDaviviendaTool`**  
  - **`name`**: `"alianzas_davivienda"`  
  - **`description`**: explica que recibe un itinerario y devuelve el mismo con recomendaciones integradas.  
  - **`_run(itinerario: str)`**:  
    1. **Scrape** de la página de Davivienda para extraer nombre, subtítulo, beneficio (descuento o 0% interés), métodos de pago y enlace de cada oferta.  
    2. Construye `alianzas_info` en formato de lista Markdown.  
    3. Llama a `chain_alianzas.run({...})` con el itinerario original y `alianzas_info`.  
    4. Registra prompt y resultado en `llm_log` mediante `log_llm`.  
    5. Devuelve el itinerario enriquecido.  

In [69]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain

# 1) Redefinir el prompt para inyectar recomendaciones dentro del mismo itinerario
prompt_alianzas = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "Usted es DaviTravel: siempre responde en español, en tercera persona, "
        "con entusiasmo y cercanía al usuario."
    ),
    HumanMessagePromptTemplate.from_template(
        "Toma este itinerario:\n\n"
        "{itinerario}\n\n"
        "Y estas alianzas activas de Davivienda en hotelería y turismo:\n"
        "{alianzas_info}\n\n"
        "Devuelve exactamente el mismo itinerario, pero **inserta** en cada sección "
        "recomendaciones concretas de productos Davivienda (tarjetas, 0% interés, descuentos) "
        "para que el usuario aproveche al máximo las alianzas: desde la compra de tiquetes, "
        "reservas de alojamiento, actividades y compras locales. "
        "Mantén la estructura original y agrega en cada punto la sugerencia correspondiente."
    )
])

# 2) Nueva cadena con el prompt ajustado
chain_alianzas = LLMChain(llm=gemini_llm, prompt=prompt_alianzas)

# 3) Herramienta actualizada (solo cambia la cadena que invoca)
class AlianzasDaviviendaTool(BaseTool):
    name: str = "alianzas_davivienda"
    description: str = (
        "Recibe el itinerario y lo devuelve con recomendaciones de Davivienda integradas."
    )

    def _run(self, itinerario: str) -> str:
        # 1) Scrape de la página de alianzas
        url = "https://ofertas.comprasdavivienda.com/categoria/hoteleria-y-turismo"
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")

        alianzas = []
        for item in soup.select("div.itemprod"):
            # Nombre y subtítulo
            name = item.select_one(".titprod")
            subtitle = item.select_one(".subtitprod")
            name = name.get_text(strip=True) if name else "Oferta"
            subtitle = subtitle.get_text(strip=True) if subtitle else ""

            # Descuento o interés
            discount = "Beneficio no especificado"
            off1 = item.select_one(".offerText .Ltext")
            off1_s = item.select_one(".offerText .Stext")
            if off1 and off1_s:
                discount = f"{off1.get_text(strip=True)} {off1_s.get_text(strip=True)}"
            else:
                off2 = item.select_one(".offerTextPercentage")
                if off2:
                    discount = off2.get_text(separator=" ", strip=True)

            # Métodos de pago aplicables
            cards = [li.get_text(strip=True) for li in item.select(".tarjetas li")]

            # Enlace y texto de CTA
            cta = item.select_one("a.cta-prod")
            href = cta["href"] if cta else ""
            cta_text = cta.get_text(strip=True) if cta else "Más info"

            alianzas.append(
                f"- **{name}**: {subtitle}. "
                f"Beneficio: {discount}. "
                f"Aplica para: {', '.join(cards)}. "
                f"[{cta_text}]({href})"
            )

        alianzas_info = "\n".join(alianzas) or "No se encontraron alianzas disponibles."

        limiter.wait()
        # 2) Generar el texto final integrando alianzas
        resultado = chain_alianzas.run({
            "itinerario": itinerario,
            "alianzas_info": alianzas_info
        })

        # 3) Registrar en el log
        log_llm(
            "Integración Alianzas Davivienda",
            prompt_alianzas.format(itinerario=itinerario, alianzas_info=alianzas_info),
            resultado
        )
        return resultado

    async def _arun(self, itinerario: str) -> str:
        return self._run(itinerario)

In [70]:
tools = [
    ExtractionTool(),
    MigratoriaTool(),
    TurismoTool(),
    CostosTool(),
    PresupuestoTool(),
    ItinerarioTool(),
    AlianzasDaviviendaTool()
]

agent = initialize_agent(
    tools=tools,
    llm=gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True
)

In [71]:
llm_log = []
consulta = construir_consulta()
limiter.wait()
result = agent.invoke({"input": consulta})
print(result["output"])
exportar_llm_log_a_excel()



> Entering new AgentExecutor chain...
Okay, I need to plan a trip from Pereira to Rome, Italy for a solo traveler interested in culture and gastronomy, during the European summer, for 4-5 days, with a budget of 5 million. I need to extract the travel requirements first.
Action: extraer_requerimientos
Action Input: Desde dónde viaja: pereira; Hacia dónde se dirige: roma, italia; Cuándo viajar: verano europa; Duración del viaje: 4 o 5 días; Tipo de viaje: cultural y gastronomico; Con quién viaja: solo; Presupuesto: 5 millones; Expectativas adicionales: quiero comer la mejor pasta
Observation: ```json
{
  "origen": "pereira",
  "destino": "roma, italia",
  "fecha": "verano europa",
  "duracion": "4 o 5 días",
  "tipo_viaje": "cultural y gastronomico",
  "acompanantes": "solo",
  "presupuesto": "5 millones",
  "expectativas": "comer la mejor pasta"
}
```
Thought:Now that I have the travel requirements, I need to check the migratory requirements for traveling from Pereira, Colombia to Rom